In [1]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from pandas import json_normalize
import pandas as pd
from rdflib import URIRef, BNode, Literal
from rdflib import Namespace
from urllib.parse import quote
import json

import psycopg2


In [2]:
### http://UPS-SCLAB-04:7200/repositories/PracticaFinal
endPoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(endPoint)

In [3]:
def get_results(query):
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)
    results = sparql.query().convert()
    return(results)

In [4]:
def receive_broaders_lists(list_broaders):

    for i in list_broaders:
      
        resource = quote(i)
        query = """
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX dct: <http://purl.org/dc/terms/>

        select distinct ?w
            {
                {VALUES (?r) {(dbc:%s)}

                    ?subject dcterms:subject ?r.
                    ?subject foaf:isPrimaryTopicOf ?w.


                FILTER (!regex(?w, "lists"))
            }
    
        }    
        """%(resource)

    resultados = get_results(query)
    df_consulta1 = json_normalize(resultados["results"]["bindings"])
    return df_consulta1 

In [5]:
# METODO PARA OBTENER EL QUERY
def get_reource_query(resource, level):

    query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>

    select distinct ?level ?r ?c1
        {
            {VALUES (?r ?level) {(dbc:""" + resource +""" + """ + str(level) +""")}
                ?r ^skos:broader{1} ?c1.

            FILTER (!regex(?c1, "lists"))
        }
    
    }    
    """

    resultados = (get_results(query))

    df_consulta1 = json_normalize(resultados["results"]["bindings"])

    return df_consulta1



In [6]:
def get_reource_query_2(resource, level):
    resource = quote(resource)
    query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>

    select distinct ?level ?r ?c1
        {
            {VALUES (?r ?level) {(dbc:%s + %s)}
                ?r ^skos:broader{1} ?c1.

            FILTER (!regex(?c1, "lists"))
        }
    
    }    
    """ %(resource,str(level))

    resultados = get_results(query)
    return resultados["results"]["bindings"]

In [7]:
resource = "Pollution"
level = 1
resultados = get_reource_query_2(resource, level)

print(type(resultados))

<class 'list'>


In [18]:
for resultado in resultados:

    #print(type(resultado))

    for key, value in resultado.items():

        #print(key,value)
    
        if key == 'c1':


            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)

        if key == 'r':
        
            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)
                    

        if key == 'level':

            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)

        
        


level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Waste
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Pollution_control_technologies
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Electronic_waste
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Species_endangered_by_pollution
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Litter
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Pollutants
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Industrial_emissions_control
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbpedia.org/resource/Category:Soil_contamination
level 1
r http://dbpedia.org/resource/Category:Pollution
c1 http://dbped

In [ ]:
'''
lista = []

for resultado in resultados:
for item in resultado:
            url = item["c1"]["value"]
                    clave = url.split(":")[-1]
                    lista.append(clave)
            json_data = json.dumps(lista)
            return JsonResponse(json_data, safe=False)

'''

In [9]:
print(type(resultados))

resultados.head()

<class 'pandas.core.frame.DataFrame'>


,level.type,level.datatype,level.value,r.type,r.value,c1.type,c1.value
0,typed-literal,http://www.w3.org/2001/XMLSchema#integer,2,uri,http://dbpedia.org/resource/Category:Pollution,uri,http://dbpedia.org/resource/Category:Waste
1,typed-literal,http://www.w3.org/2001/XMLSchema#integer,2,uri,http://dbpedia.org/resource/Category:Pollution,uri,http://dbpedia.org/resource/Category:Pollution...
2,typed-literal,http://www.w3.org/2001/XMLSchema#integer,2,uri,http://dbpedia.org/resource/Category:Pollution,uri,http://dbpedia.org/resource/Category:Electroni...
3,typed-literal,http://www.w3.org/2001/XMLSchema#integer,2,uri,http://dbpedia.org/resource/Category:Pollution,uri,http://dbpedia.org/resource/Category:Species_e...
4,typed-literal,http://www.w3.org/2001/XMLSchema#integer,2,uri,http://dbpedia.org/resource/Category:Pollution,uri,http://dbpedia.org/resource/Category:Litter


In [10]:
# Filter the columns that are of interest.
resultados = resultados[['level.value', 'r.value', 'c1.value']]

resultados.head(3)


,level.value,r.value,c1.value
0,2,http://dbpedia.org/resource/Category:Pollution,http://dbpedia.org/resource/Category:Waste
1,2,http://dbpedia.org/resource/Category:Pollution,http://dbpedia.org/resource/Category:Pollution...
2,2,http://dbpedia.org/resource/Category:Pollution,http://dbpedia.org/resource/Category:Electroni...


In [11]:
# Assuming your DataFrame is named df and you want to transform the 'column_name' column
column_list = resultados['c1.value'].tolist()

# Display the resulting list
print(column_list)

['http://dbpedia.org/resource/Category:Waste', 'http://dbpedia.org/resource/Category:Pollution_control_technologies', 'http://dbpedia.org/resource/Category:Electronic_waste', 'http://dbpedia.org/resource/Category:Species_endangered_by_pollution', 'http://dbpedia.org/resource/Category:Litter', 'http://dbpedia.org/resource/Category:Pollutants', 'http://dbpedia.org/resource/Category:Industrial_emissions_control', 'http://dbpedia.org/resource/Category:Soil_contamination', 'http://dbpedia.org/resource/Category:Radioactive_contamination', 'http://dbpedia.org/resource/Category:Water_pollution', 'http://dbpedia.org/resource/Category:Ocean_pollution', 'http://dbpedia.org/resource/Category:Noise_pollution', 'http://dbpedia.org/resource/Category:Pollution_by_country', 'http://dbpedia.org/resource/Category:Space_debris', 'http://dbpedia.org/resource/Category:Chemical_accident', 'http://dbpedia.org/resource/Category:Light_pollution', 'http://dbpedia.org/resource/Category:Air_pollution', 'http://dbp

In [6]:
lista = ["Waste","Pollution_control_technologies","Electronic_waste"]
resultados = receive_broaders_lists(lista)

In [7]:
print(type(resultados))

<class 'pandas.core.frame.DataFrame'>


In [8]:
resultados

,w.type,w.value
0,uri,http://en.wikipedia.org/wiki/Electronic_waste
1,uri,http://en.wikipedia.org/wiki/E-Stewards
2,uri,http://en.wikipedia.org/wiki/Techreturns
3,uri,http://en.wikipedia.org/wiki/Global_waste_trade
4,uri,http://en.wikipedia.org/wiki/Closing_the_Loop
5,uri,http://en.wikipedia.org/wiki/Computer_liquidator
6,uri,http://en.wikipedia.org/wiki/Demanufacturing
7,uri,http://en.wikipedia.org/wiki/Solving_the_E-was...
8,uri,http://en.wikipedia.org/wiki/MyGreenElectronics
9,uri,http://en.wikipedia.org/wiki/Disposable_camera


In [11]:
# Filter the columns that are of interest.
resultados = resultados[['level.value', 'r.value', 'c1.value']]

resultados.head(3)

,level.value,r.value,c1.value
0,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Computer_...
1,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Electroni...
2,3,http://dbpedia.org/resource/Category:Electroni...,http://dbpedia.org/resource/Category:Electroni...


In [12]:
# Assuming your DataFrame is named df and you want to transform the 'column_name' column
column_list = resultados['c1.value'].tolist()

# Display the resulting list
print(column_list)


['http://dbpedia.org/resource/Category:Computer_recycling', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Africa', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Europe', 'http://dbpedia.org/resource/Category:Electronic_waste_in_Asia', 'http://dbpedia.org/resource/Category:Electronic_waste_by_country']


## Conexión base de datos para enviar los datos

In [ ]:
def get_reource_query_2(resource, level):
    resource = quote(resource)
    query = """
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dct: <http://purl.org/dc/terms/>

    select distinct ?level ?r ?c1
        {
            {VALUES (?r ?level) {(dbc:%s + %s)}
                ?r ^skos:broader{1} ?c1.

            FILTER (!regex(?c1, "lists"))
        }
    
    }    
    """ %(resource,str(level))

    resultados = get_results(query)
    return resultados["results"]["bindings"]

In [ ]:
resource = "Pollution"
level = 1
resultados = get_reource_query_2(resource, level)

print(type(resultados))

In [ ]:
for resultado in resultados:

    #print(type(resultado))

    for key, value in resultado.items():

        #print(key,value)
    
        if key == 'c1':


            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)

        if key == 'r':
        
            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)
                    

        if key == 'level':

            for key_2, value_2 in value.items():

                #print(key_2,value_2)
                if key_2 == 'value':

                    print(key,value_2)

In [3]:
conn = psycopg2.connect(database="data_concepts",
                        host="localhost",
                        user="fico",
                        password="password",
                        port="5432")

In [5]:
cursor = conn.cursor()

In [6]:
print(cursor.fetchone())


ProgrammingError: no results to fetch